In [1]:
import json
import math
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("ml-100k/u.data", header=None, sep="\t")
print(data.shape)
data.columns = ["user_id", "item_id", "rating", "timestamp"]
data.head()

(100000, 4)


,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [3]:
item = pd.read_csv("ml-100k/u.item", header=None, sep="|", encoding='latin-1')
item.columns = ["item_id","movie_title", "release_date", "video_release_date", \
              "IMDb_URL", "unknown", "Action", "Adventure", "Animation", \
              "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", \
              "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", \
              "Thriller", "War", "Western"]
print(item.shape)
item.head()

(1682, 24)


,item_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


1 пункт <br />
Построить распределения оценок. Получится таблица, где в первом столбце стоят значения оценки 1,2,3,4,5, а во втором — количество человек, поставивших соответствующую оценку. Поле должно называться “hist_film”.

In [4]:
data.rating.unique()

array([3, 1, 2, 4, 5])

In [5]:
data_gr = data[data["item_id"] == 69][["user_id","rating"]].groupby("rating").count().reset_index()
data_gr.columns = ["rating","count"]
data_gr

,rating,count
0,1,14
1,2,23
2,3,59
3,4,125
4,5,100


2 пункт <br />
Чтобы понять какие оценки предпочитают ставить пользователи MovieLens, постройте аналогичное предыдущему пункту распределение оценок, но только уже для всего датасета. Поле должно называться “hist_all”.

In [6]:
data_gr_2 = data[["user_id","rating"]].groupby("rating").count().reset_index()
data_gr_2.columns = ["rating","count"]
data_gr_2

,rating,count
0,1,6110
1,2,11370
2,3,27145
3,4,34174
4,5,21201


In [7]:
output_1 = []
for index, row in data_gr.iterrows():
    output_1.append(int(row["count"]))
output_1

[14, 23, 59, 125, 100]

In [8]:
output_2 = []
for index, row in data_gr_2.iterrows():
    output_2.append(int(row["count"]))
output_2

[6110, 11370, 27145, 34174, 21201]

In [9]:
output = {"hist_film": output_1, "hist_all": output_2}
output

{'hist_film': [14, 23, 59, 125, 100],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

In [10]:
json_data = json.dumps(output)
json_data

'{"hist_film": [14, 23, 59, 125, 100], "hist_all": [6110, 11370, 27145, 34174, 21201]}'

In [11]:
with open('../lab06.json', 'w') as f:
    json.dump(output, f)

*************Суперачика*************

1. Количество человек n, поставивших рейтинг фильму

In [12]:
cnt_person_per_film = data[["user_id","item_id"]].groupby("item_id").count().reset_index()
cnt_person_per_film.head()

,item_id,user_id
0,1,452
1,2,131
2,3,90
3,4,209
4,5,86


2. Средний рейтинг фильма (сумма рейтингов фильма / количество человек, оценивших фильм)

In [13]:
avg_rating = data[["item_id","user_id","rating"]].groupby("item_id").agg({"user_id":"count","rating":"sum"}).reset_index()
avg_rating["avg_rating"] = avg_rating["rating"] / avg_rating["user_id"]
avg_rating.head()

,item_id,user_id,rating,avg_rating
0,1,452,1753,3.878319
1,2,131,420,3.206107
2,3,90,273,3.033333
3,4,209,742,3.550239
4,5,86,284,3.302326


3. Количество человек m, оценивших фильм положительно. Оценки 4 и выше считаются положительными.

In [14]:
cnt_person_per_film_positive = data[data["rating"]>=4][["user_id","item_id"]].groupby("item_id").count().reset_index()
cnt_person_per_film_positive.head()

,item_id,user_id
0,1,321
1,2,51
2,3,34
3,4,122
4,5,39


4. Доля людей, оценивших фильм положительно (пункт 3 / пункт 1 или m / n).

In [15]:
positive_all = pd.merge(cnt_person_per_film_positive, avg_rating, on='item_id')
positive_all["pos_all"] = positive_all["user_id_x"] / positive_all["user_id_y"]
positive_all.head()

,item_id,user_id_x,user_id_y,rating,avg_rating,pos_all
0,1,321,452,1753,3.878319,0.710177
1,2,51,131,420,3.206107,0.389313
2,3,34,90,273,3.033333,0.377778
3,4,122,209,742,3.550239,0.583732
4,5,39,86,284,3.302326,0.453488


5. Глобальное среднее по всему датасету. Сумма всех оценок по всем фильмам /Количество всех оценок по всем фильмам.

In [16]:
mu = data["rating"].sum() / data["user_id"].count()
k = 30
mu, k

(3.52986, 30)

6. Оценку, поправленную на нехватку данных:

In [17]:
avg_rating_corr = data[["item_id","user_id","rating"]].groupby("item_id").agg({"user_id":"count","rating":"sum"}).reset_index()
avg_rating_corr["avg_rating"] = (avg_rating_corr["rating"] + k*mu) / (avg_rating_corr["user_id"] + k)
avg_rating_corr.head()

,item_id,user_id,rating,avg_rating
0,1,452,1753,3.856630
1,2,131,420,3.266434
2,3,90,273,3.157465
3,4,209,742,3.547681
4,5,86,284,3.361171


7. Нижнюю и верхнюю границы доверительного интервала оценки (Wilson score interval) из лекции с заданным уровнем доверия

In [36]:
z = 3.291
df = pd.merge(cnt_person_per_film,positive_all,on="item_id")
df = df[["item_id","user_id","pos_all"]]
df["z_2"] = np.power(z,2)
df["n_2"] = np.power(df["user_id"],2)
df["sqrt"] = z*np.sqrt(((df["pos_all"]*(1-df["pos_all"]))/df["user_id"])+(df["z_2"]/(4*df["n_2"])))

df.head()

,item_id,user_id,pos_all,z_2,n_2,sqrt
0,1,452,0.710177,10.830681,204304,0.071242
1,2,131,0.389313,10.830681,17161,0.146168
2,3,90,0.377778,10.830681,8100,0.178628
3,4,209,0.583732,10.830681,43681,0.115167
4,5,86,0.453488,10.830681,7396,0.187556


In [37]:
df["high_level"] = (1 / (1 + (df["z_2"]/df["user_id"])))*(df["pos_all"] + (df["z_2"]/(2*df["user_id"]))+df["sqrt"])
df["low_level"] = (1 / (1 + (df["z_2"]/df["user_id"])))*(df["pos_all"] + (df["z_2"]/(2*df["user_id"]))-df["sqrt"])
df.head()

,item_id,user_id,pos_all,z_2,n_2,sqrt,high_level,low_level
0,1,452,0.710177,10.830681,204304,0.071242,0.774834,0.635683
1,2,131,0.389313,10.830681,17161,0.146168,0.532772,0.262759
2,3,90,0.377778,10.830681,8100,0.178628,0.550347,0.231465
3,4,209,0.583732,10.830681,43681,0.115167,0.689099,0.470114
4,5,86,0.453488,10.830681,7396,0.187556,0.625268,0.292113


Подготовка ответа

In [38]:
ans_1 = pd.merge(cnt_person_per_film, item, on="item_id")
ans_1 = ans_1[["item_id","movie_title","user_id"]].sort_values(["user_id","movie_title"],ascending=[False, True]).head(10)
my_list_1 = []
for index, row in ans_1.iterrows():
    my_list_1.append(int(row["item_id"]))
ans_1 = {"top10_rates": my_list_1}
ans_1

{'top10_rates': [50, 258, 100, 181, 294, 286, 288, 1, 300, 121]}

In [39]:
ans_2 = pd.merge(avg_rating, item, on="item_id")
ans_2 = ans_2[["item_id","movie_title","avg_rating"]].sort_values(["avg_rating","movie_title"],ascending=[False, True]).head(10)
my_list_2 = []
for index, row in ans_2.iterrows():
    my_list_2.append(int(row["item_id"]))
ans_2 = {"top10_average": my_list_2}
ans_2

{'top10_average': [1536, 1653, 814, 1201, 1189, 1467, 1500, 1599, 1293, 1122]}

In [40]:
ans_3 = pd.merge(avg_rating_corr, item, on="item_id")
ans_3 = ans_3[["item_id","movie_title","avg_rating"]].sort_values(["avg_rating","movie_title"],ascending=[False, True]).head(10)
my_list_3 = []
for index, row in ans_3.iterrows():
    my_list_3.append(int(row["item_id"]))
ans_3 = {"top10_rating": my_list_3}
ans_3

{'top10_rating': [318, 64, 483, 50, 12, 408, 603, 169, 98, 127]}

In [41]:
ans_4 = pd.merge(df, item, on="item_id")
ans_4 = ans_4[["item_id","movie_title","low_level"]].sort_values(["low_level","movie_title"],ascending=[False, True]).head(10)
my_list_4 = []
for index, row in ans_4.iterrows():
    my_list_4.append(int(row["item_id"]))
ans_4 = {"top10_lower": my_list_4}
ans_4

{'top10_lower': [64, 98, 318, 479, 50, 483, 603, 427, 357, 12]}

In [42]:
output = {"top10_rates":my_list_1, "top10_average": my_list_2, "top10_rating": my_list_3, "top10_lower": my_list_4}
output

{'top10_rates': [50, 258, 100, 181, 294, 286, 288, 1, 300, 121],
 'top10_average': [1536, 1653, 814, 1201, 1189, 1467, 1500, 1599, 1293, 1122],
 'top10_rating': [318, 64, 483, 50, 12, 408, 603, 169, 98, 127],
 'top10_lower': [64, 98, 318, 479, 50, 483, 603, 427, 357, 12]}

In [43]:
json_data = json.dumps(output)
json_data

'{"top10_rates": [50, 258, 100, 181, 294, 286, 288, 1, 300, 121], "top10_average": [1536, 1653, 814, 1201, 1189, 1467, 1500, 1599, 1293, 1122], "top10_rating": [318, 64, 483, 50, 12, 408, 603, 169, 98, 127], "top10_lower": [64, 98, 318, 479, 50, 483, 603, 427, 357, 12]}'

In [44]:
with open('../lab06s.json', 'w') as f:
    json.dump(output, f)